In [1]:
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import os # for creating directories

Using TensorFlow backend.


In [2]:
# env = gym.make('MountainCar-v0') # initialise environment
env = gym.make('CartPole-v0')
state_size = env.observation_space.shape[0]
action_size = env.action_space.n
batch_size = 32
n_episodes = 1001 # n games we want agent to play (default 1001)
# output_dir = 'model_output/mountaincar/'
output_dir = 'model_output/cartpole/'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [3]:
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = deque(maxlen=2000) 
        self.epsilon = 1.0
        self.model = self._build_model() 
    
    def _build_model(self):
        model = Sequential()
        model.add(Dense(24, input_dim=self.state_size, activation='relu')) 
        model.add(Dense(24, activation='relu'))
        model.add(Dense(self.action_size, activation='linear')) 
        model.compile(loss='mse',optimizer=Adam(lr=0.001))
        return model
    
    def remember(self, state, action, reward, next_state, done):
        self.memory.append((state, action, reward, next_state, done))
        
        
    def act(self, state):
        
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        else:
            act_values = self.model.predict(state) 
        
        return np.argmax(act_values[0])

    def replay(self, batch_size): 
        
        minibatch = random.sample(self.memory, batch_size) 
#         minibatch = self.memory 
        
        for state, action, reward, next_state, done in minibatch: 
        
            if done:
                target = reward
            else: 
                target = reward + 0.97 * np.amax(self.model.predict(next_state)[0])
            
            target_f = self.model.predict(state)
            target_f[0][action] = target
        
            self.model.fit(state, target_f, epochs=1, verbose=0)
        
        if self.epsilon > 0.01:
            self.epsilon = self.epsilon * 0.995

In [4]:
agent = DQNAgent(state_size, action_size) # initialise agent
## done = False
for e in range(n_episodes): 
    state = env.reset()
    state = np.reshape(state, [1, state_size])
    max_position = -.4
    for time in range(500): 
#         env.render()
        action = agent.act(state)
        next_state, reward, done, _ = env.step(action)
        next_state = np.reshape(next_state, [1, state_size])
        agent.remember(state, action, reward, next_state, done) 
        state = next_state   
        
        if done:
            print("episode: {}/{}, score: {}, e: {:.2}".format(e, n_episodes, time, agent.epsilon))
            break
            
    if len(agent.memory) > batch_size:
        agent.replay(batch_size) 

Instructions for updating:
Colocations handled automatically by placer.
episode: 0/1001, score: 38, e: 1.0
Instructions for updating:
Use tf.cast instead.
episode: 1/1001, score: 31, e: 0.99
episode: 2/1001, score: 34, e: 0.99
episode: 3/1001, score: 16, e: 0.99
episode: 4/1001, score: 8, e: 0.98
episode: 5/1001, score: 11, e: 0.98
episode: 6/1001, score: 28, e: 0.97
episode: 7/1001, score: 13, e: 0.97
episode: 8/1001, score: 8, e: 0.96
episode: 9/1001, score: 23, e: 0.96
episode: 10/1001, score: 22, e: 0.95
episode: 11/1001, score: 8, e: 0.95
episode: 12/1001, score: 14, e: 0.94
episode: 13/1001, score: 19, e: 0.94
episode: 14/1001, score: 10, e: 0.93
episode: 15/1001, score: 20, e: 0.93
episode: 16/1001, score: 17, e: 0.92
episode: 17/1001, score: 12, e: 0.92
episode: 18/1001, score: 15, e: 0.91
episode: 19/1001, score: 11, e: 0.91
episode: 20/1001, score: 17, e: 0.9
episode: 21/1001, score: 41, e: 0.9
episode: 22/1001, score: 17, e: 0.9
episode: 23/1001, score: 13, e: 0.89
episode: 

KeyboardInterrupt: 